In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np

import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.colors import ListedColormap

import networkx as nx

from minimal.imaging import blit_rooms
from minimal.layout import LayoutGraph, NodeType
from minimal.gen import generate_plan, PlanMasks
from minimal import sample_graphs
from minimal.rooms import RoomAreas, extract_rooms
from minimal.walls import intersect_rooms, find_walls

%matplotlib inline

In [8]:
%%time
pm = PlanMasks.create_from_state(torch.load("data/plan_masks_01.pth"))
rooms = extract_rooms(pm)

CPU times: user 1.31 s, sys: 2.58 ms, total: 1.31 s
Wall time: 1.31 s
